In [ ]:
import configparser
import random

import numpy as np
import torch
from llama_cpp import Llama
from transformers import set_seed

In [ ]:
SEED_NUMBER = 42

random.seed(SEED_NUMBER)
np.random.seed(SEED_NUMBER)
torch.manual_seed(SEED_NUMBER)
set_seed(SEED_NUMBER)

In [ ]:
config_directory = "../properties/local.ini"
config = configparser.ConfigParser()
config.read(config_directory)

rephrasing_model_directory = config["REPHRASING_MODEL"]["model_directory"]

In [ ]:
rephrasing_model_directory

In [ ]:
rephrasing_model = Llama(
    model_path=rephrasing_model_directory,
    n_batch=512,
    n_gpu_layers=-1,
    n_ctx=2048,
    verbose=False,
)

In [ ]:
rephrasing_prompt = """
=================== Talimatlar ===================
- Asagidaki cümleyi soru sözcükleri ekleyerek formatinda soruya dönüştürmelisin.
- Cümle zaten bir soruysa, olduğu gibi bırak ve herhangi bir değişiklik yapma.
- Yanıtta, dönüştürülmüş cümleyi not veya öneri olmadan ver.
- Birden fazla soru varsa, yalnızca dönüştürülmüş soruları nokta işareti olmadan dönüştür.
- Belirli bir cümlenin iki farklı soru versiyonu varsa, yalnızca birini ver.
- Dönüşüm için aynı kelimeleri kullan, sadece ekstra soru sözcükleri ekle.
- Aşağıdaki örnekleri kullanabilirsin.

=================== Örnekler ===================
örnek: Seftali suyu
istenen versiyon: Seftali suyu nedir?

örnek: Bilgisayarin batarya sarji.
istenen versiyon: Bilgisayarin bataryasini ne sarj eder?

örnek: Fransa'nin baskenti neresidir?
istenen versiyon: Fransa'nin baskenti neresidir?
=================================================

<|im_start|>user
{question}<|im_end|>
<|im_start|>assistant
"""

# rephrasing_prompt = """
# =================== Talimatlar ===================
# - Asagidaki cümleyi soru sözcükleri ekleyerek formatinda soruya dönüştürmelisin.
# - Cümle zaten bir soruysa, olduğu gibi bırak ve herhangi bir değişiklik yapma.
# - Yanıtta, dönüştürülmüş cümleyi not veya öneri olmadan ver.
# - Birden fazla soru varsa, yalnızca dönüştürülmüş soruları nokta işareti olmadan dönüştür.
# - Belirli bir cümlenin iki farklı soru versiyonu varsa, yalnızca birini ver.
# - Dönüşüm için aynı kelimeleri kullan, sadece ekstra soru sözcükleri ekle.
# - Aşağıdaki örnekleri kullanabilirsin.

# =================== Örnekler ===================
# örnek: Seftali suyu
# istenen versiyon: Seftali suyu nedir?

# örnek: Bilgisayarin batarya sarji.
# istenen versiyon: Bilgisayarin bataryasini ne sarj eder?

# örnek: Fransa'nin baskenti neresidir?
# istenen versiyon: Fransa'nin baskenti neresidir?
# =================================================

# <|begin_of_text|>user
# {question}<|eot_id|>"
# <|begin_of_text|>assistant
# """

In [ ]:
def transform_sentence(model: Llama, prompt: str) -> str:
    output = model(
        prompt=prompt,
        temperature=0.2,
        repeat_penalty=1.2,
        max_tokens=1024,
        stop=[
            "<|im_end|>",
            "<|eot_id|>",
            "<|end_of_text|>",
            "\n<|im_start|>",
            "\n<|begin_of_text|>",
            "istenen versiyon",
            "==",
            "Human:",
            "Assistant:",
        ],
    )
    generated_answer: str = output["choices"][0]["text"]
    return generated_answer

In [ ]:
questions = [
    "modelin turkce dil destegi",
    "paris'teki en kalabalik muzenin bilet fiyati",
    "kafeinsiz kahve cesitleri",
    "t-shirt'un buyuk bedeni",
    "orhan pamuk cok unlur bir yazar. onun en cok satan romani",
    "pantolonun kaliplari",
    "4 kapili arabalarin fiyatlari ile 2 kapili arabalarin fiyatlari farki",
    "istanbul'un en pahali semti neresidir?",
    "samsung en iyi televizyonu lg'nin en iyi televizyonu",
    "icecekler (alkollu)",
]

In [ ]:
for question in questions:
    rephrase_prompt = rephrasing_prompt.format(question=question)
    rephrased_question = transform_sentence(
        model=rephrasing_model, prompt=rephrase_prompt
    )
    print("Orijinal Cumle: ")
    print(question)
    print()
    print("Yeniden Ifade Edilmis Cumle: ")
    print(rephrased_question)
    print()